RANDOM FOREST CLASSIFICATION

Random Forest is similar to Bagging using multiple model versions and aggregating the ensemble of models to make a single prediction. RF uses an ensemble of tree’s and introduces randomness into each tree by randomly selecting a subset of the features for each node to split on. This makes the predictions of each tree uncorrelated, improving results when the models are aggregated. 

motive:Sampling Process of Random Forest to Bagging, then demonstrate how each predictor for random forest is not correlated


In [2]:
!pip install pandas
!pip install numpy
!pip install matplotlib
!pip install seaborn
!pip install --upgrade scikit-learn

In [3]:
# importing all the necessary libraries
import pandas as pd
import pylab as plt
import numpy as np
import scipy.optimize as opt
from sklearn import preprocessing
%matplotlib inline
import matplotlib.pyplot as plt
from sklearn import metrics
from tqdm import tqdm

In [4]:
# ignore error
import warnings
warnings.filterwarnings('ignore')


In [5]:
# function to calculate accuracy
def get_accuracy(X_train,X_test,y_train,y_test,model):
    return {"test Accuracy":metrics.accuracy_score(y_test,model.predict(X_test)),"train Accuracy":metrics.accuracy_score(y_train,model.predict(X_train))}

In [45]:
# calculate average correlation between predictors and display pairwise correlation between predictors
def get_correlation(X_test,y_test,model):
    n_estimators=len(model.estimators_) #get the number of estimators
    prediction=np.zeros((y_test.shape[0],n_estimators))
    predictions=pd.DataFrame ({'estimator'+str(n+1):[] for n in range (n_estimators)}) #initialize empty dataframe for predictions
    # make predictions using each estimator:
    for key,model in zip(predictions.keys(),model.estimators_):
        predictions[key]=model.predict(X_test.to_numpy())
    corr=predictions.corr() #compute correlation matrix
    print("Average correlation between predictors",corr.mean().mean()-1/n_estimators) #substract 1/n_estimators to avoid bias
    return corr



RF is similar to Bagging in that it uses model ensembles to make predictions. Unlike Bagging, when you add more models, RF does not suffer from Overfitting.

The dataset includes information about:

*   Customers who left within the last month – the column is called Churn
*   Services that each customer has signed up for – phone, multiple lines, internet, online security, online backup, device protection, tech support, and streaming TV and movies
*   Customer account information – how long they had been a customer, contract, payment method, paperless billing, monthly charges, and total charges
*   Demographic info about customers – gender, age range, and if they have partners and dependents



In [7]:
# Load the Data 
churn_df=pd.read_csv("E:\AI-ML\Supervised Machine Learning Classification\Random Forest\ChurnData.csv")


In [8]:
churn_df.head()

,tenure,age,address,income,ed,employ,equip,callcard,wireless,longmon,tollmon,equipmon,cardmon,wiremon,longten,tollten,cardten,voice,pager,internet,callwait,confer,ebill,loglong,logtoll,lninc,custcat,churn
0,11.0,33.0,7.0,136.0,5.0,5.0,0.0,1.0,1.0,4.40,20.75,0.0,15.25,35.7,42.00,211.45,125.0,1.0,1.0,0.0,1.0,1.0,0.0,1.482,3.033,4.913,4.0,1.0
1,33.0,33.0,12.0,33.0,2.0,0.0,0.0,0.0,0.0,9.45,0.00,0.0,0.00,0.0,288.80,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.246,3.240,3.497,1.0,1.0
2,23.0,30.0,9.0,30.0,1.0,2.0,0.0,0.0,0.0,6.30,0.00,0.0,0.00,0.0,157.05,0.00,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.841,3.240,3.401,3.0,0.0
3,38.0,35.0,5.0,76.0,2.0,10.0,1.0,1.0,1.0,6.05,45.00,50.1,23.25,64.9,239.55,1873.05,880.0,1.0,1.0,1.0,1.0,1.0,1.0,1.800,3.807,4.331,4.0,0.0
4,7.0,35.0,14.0,80.0,2.0,15.0,0.0,1.0,0.0,7.10,22.00,0.0,23.75,0.0,47.45,166.10,145.0,1.0,0.0,0.0,1.0,1.0,0.0,1.960,3.091,4.382,3.0,0.0


In [ ]:
# data pre processing
churn_df=churn_df[['tenure','age','address','income','ed','employ','equip','callcard','wireless','churn']]
churn_df['churn']=churn_df['churn'].astype('int')
churn_df.head()
# we have removed the floating factor in churn

,tenure,age,address,income,ed,employ,equip,callcard,wireless,churn
0,11.0,33.0,7.0,136.0,5.0,5.0,0.0,1.0,1.0,1
1,33.0,33.0,12.0,33.0,2.0,0.0,0.0,0.0,0.0,1
2,23.0,30.0,9.0,30.0,1.0,2.0,0.0,0.0,0.0,0
3,38.0,35.0,5.0,76.0,2.0,10.0,1.0,1.0,1.0,0
4,7.0,35.0,14.0,80.0,2.0,15.0,0.0,1.0,0.0,0


Bootstrap Sampling

Bootstrap Sampling is a method that involves drawing sample data repeatedly with replacement from a data source to estimate a model parameter. Scikit-learn has methods for Bagging, but it is helpful to understand Bootstrap Sampling. We will import "resample".


In [10]:
from sklearn.utils import resample

In [11]:
churn_df[0:5]

,tenure,age,address,income,ed,employ,equip,callcard,wireless,churn
0,11.0,33.0,7.0,136.0,5.0,5.0,0.0,1.0,1.0,1
1,33.0,33.0,12.0,33.0,2.0,0.0,0.0,0.0,0.0,1
2,23.0,30.0,9.0,30.0,1.0,2.0,0.0,0.0,0.0,0
3,38.0,35.0,5.0,76.0,2.0,10.0,1.0,1.0,1.0,0
4,7.0,35.0,14.0,80.0,2.0,15.0,0.0,1.0,0.0,0


In [12]:
# do bootstrap sample 
for n in range(5):
    print(resample(churn_df[0:5]))

   tenure   age  address  income   ed  employ  equip  callcard  wireless  churn
0    11.0  33.0      7.0   136.0  5.0     5.0    0.0       1.0       1.0      1
0    11.0  33.0      7.0   136.0  5.0     5.0    0.0       1.0       1.0      1
0    11.0  33.0      7.0   136.0  5.0     5.0    0.0       1.0       1.0      1
2    23.0  30.0      9.0    30.0  1.0     2.0    0.0       0.0       0.0      0
3    38.0  35.0      5.0    76.0  2.0    10.0    1.0       1.0       1.0      0
   tenure   age  address  income   ed  employ  equip  callcard  wireless  churn
2    23.0  30.0      9.0    30.0  1.0     2.0    0.0       0.0       0.0      0
2    23.0  30.0      9.0    30.0  1.0     2.0    0.0       0.0       0.0      0
4     7.0  35.0     14.0    80.0  2.0    15.0    0.0       1.0       0.0      0
1    33.0  33.0     12.0    33.0  2.0     0.0    0.0       0.0       0.0      1
4     7.0  35.0     14.0    80.0  2.0    15.0    0.0       1.0       0.0      0
   tenure   age  address  income   ed  e

In [13]:
# we see that the dataset is of same size(5) but some rows are repeted

Like Bagging, RF uses an independent bootstrap sample from the training data. In addition, we select $m$ variables at random out of all M possible
variables. Let's do an example.


In [14]:
# set variables at random

In [16]:
X=churn_df[['tenure','age','address','income','ed','employ','equip']]

In [24]:
M=X.shape[1]
M
m=3

In [20]:
# lets select m=3 and randomly sample feautes from 5 bootsrap  samples from above

In [21]:
feature_index=range(M)
feature_index

range(0, 7)

In [22]:
# to randomly select indexes

In [27]:
import random
random.sample(feature_index,m)
# randomly 3 features select hojaeeghe

[6, 1, 3]

In [29]:
# from each node we select random index
for n in range(5):
    print("sample{}".format(n))
    print(resample(X[0:5]).iloc[:,random.sample(feature_index,m)])

sample0
   address   ed  employ
0      7.0  5.0     5.0
1     12.0  2.0     0.0
4     14.0  2.0    15.0
3      5.0  2.0    10.0
3      5.0  2.0    10.0
sample1
   tenure  address  income
4     7.0     14.0    80.0
0    11.0      7.0   136.0
3    38.0      5.0    76.0
2    23.0      9.0    30.0
4     7.0     14.0    80.0
sample2
    ed   age  employ
3  2.0  35.0    10.0
2  1.0  30.0     2.0
2  1.0  30.0     2.0
4  2.0  35.0    15.0
1  2.0  33.0     0.0
sample3
   equip  tenure   ed
2    0.0    23.0  1.0
4    0.0     7.0  2.0
1    0.0    33.0  2.0
4    0.0     7.0  2.0
2    0.0    23.0  1.0
sample4
   equip  income   ed
0    0.0   136.0  5.0
2    0.0    30.0  1.0
4    0.0    80.0  2.0
4    0.0    80.0  2.0
2    0.0    30.0  1.0


TRAIN/TEST DATASET

In [30]:
y=churn_df['churn']
y.head()

0    1
1    1
2    0
3    0
4    0
Name: churn, dtype: int32

In [31]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,train_size=0.3,random_state=1)
print("Train set",X_train.shape,y_train.shape)
print("Test set",X_test.shape, y_test.shape)

Train set (60, 7) (60,)
Test set (140, 7) (140,)


In [32]:
# USING BAGGING

In [33]:
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier

Bagging improves models that suffer from overfitting; they do well on the training data, but they do not generalize well to unseen data. Decision Trees are a prime candidate for this reason. In addition, they are fast to train; We create a BaggingClassifier object,  with a Decision Tree as the estimator.


In [35]:
n_estimators=20
Bag= BaggingClassifier(estimator=DecisionTreeClassifier(criterion="entropy",max_depth=4,random_state=2),n_estimators=n_estimators,random_state=0,bootstrap=True)


In [37]:
# fit the model
Bag.fit(X_train,y_train)

BaggingClassifier(estimator=DecisionTreeClassifier(criterion='entropy',
                                                   max_depth=4,
                                                   random_state=2),
                  n_estimators=20, random_state=0)

In [39]:
# predict
Bag.predict(X_test).shape

(140,)

In [40]:
print(get_accuracy(X_train,X_test,y_train,y_test,Bag))

{'test Accuracy': 0.75, 'train Accuracy': 0.95}


In [47]:
get_correlation(X_test, y_test,Bag).style.background_gradient(cmap='coolwarm')


Average correlation between predictors 0.1984800367900587


,estimator1,estimator2,estimator3,estimator4,estimator5,estimator6,estimator7,estimator8,estimator9,estimator10,estimator11,estimator12,estimator13,estimator14,estimator15,estimator16,estimator17,estimator18,estimator19,estimator20
estimator1,1.000000,0.108099,0.089135,0.250878,0.273909,0.353929,0.328585,0.107618,0.198809,0.356710,0.288225,0.212856,0.049635,0.143611,0.204815,0.271110,0.230021,0.371457,0.233169,0.222455
estimator2,0.108099,1.000000,0.189029,0.740818,0.391040,-0.012413,0.053347,0.343303,0.029440,0.188448,0.148200,0.259889,0.028959,0.162556,0.203340,-0.091368,0.382684,0.110286,0.197555,0.281351
estimator3,0.089135,0.189029,1.000000,0.295965,0.273909,0.125738,0.044842,0.289143,0.125176,0.388846,0.109283,-0.133206,0.019811,-0.193011,0.449631,-0.019736,0.291772,0.209653,-0.031919,0.492226
estimator4,0.250878,0.740818,0.295965,1.000000,0.330402,0.006915,0.065084,0.244530,0.032803,0.281556,0.081299,0.283458,0.081299,0.082123,0.173010,-0.078667,0.323237,0.145196,0.103634,0.270568
estimator5,0.273909,0.391040,0.273909,0.330402,1.000000,0.076139,0.194675,0.138675,0.165531,0.216730,0.297785,0.056130,0.267310,0.328786,0.485138,-0.152845,0.858151,0.328787,0.312891,0.381554
estimator6,0.353929,-0.012413,0.125738,0.006915,0.076139,1.000000,0.614730,0.473319,0.158114,0.379536,0.196698,0.241805,-0.027446,0.178362,0.211219,0.334575,0.066299,0.392119,0.348508,0.151718
estimator7,0.328585,0.053347,0.044842,0.065084,0.194675,0.614730,1.000000,0.390274,0.206437,0.213979,0.415828,0.290359,0.102277,0.239768,0.204020,0.057030,0.149680,0.412861,0.674313,0.188332
estimator8,0.107618,0.343303,0.289143,0.244530,0.138675,0.473319,0.390274,1.000000,-0.029348,0.249769,0.096370,0.320200,-0.052218,-0.050967,0.192978,0.049682,0.218435,0.199463,0.300801,0.328974
estimator9,0.198809,0.029440,0.125176,0.032803,0.165531,0.158114,0.206437,-0.029348,1.000000,0.374088,0.368869,-0.000000,0.224214,0.252330,0.326545,0.246871,0.157243,0.384553,0.312255,0.058155
estimator10,0.356710,0.188448,0.388846,0.281556,0.216730,0.379536,0.213979,0.249769,0.374088,1.000000,0.230433,0.020349,0.041036,0.184623,0.443742,0.323230,0.336596,0.469247,0.128263,0.440990


RANDOM FOREST


Random forests are a combination of trees such that each tree depends on a random subset of the features and data. As a result, each tree in the forest is different and usually performs better than Bagging. The most important parameters are the number of trees and the number of features to sample. First, we import RandomForestClassifier.


In [48]:
from sklearn.ensemble import RandomForestClassifier

In [49]:
n_estimators=20

In [50]:
# max _features m is the number of features we consider when looking for best split 
M_features=X.shape[1]

In [51]:
# to calculate m

In [52]:
# m=floor(underoot M)
max_features=round(np.sqrt(M_features))-1
max_features

2

In [53]:
y_test

58     1
40     0
34     0
102    0
184    0
      ..
23     0
126    0
9      0
170    0
104    0
Name: churn, Length: 140, dtype: int32

In [54]:
model=RandomForestClassifier(max_features=max_features,n_estimators=n_estimators,random_state=0)

In [55]:
model.fit(X_train,y_train)

RandomForestClassifier(max_features=2, n_estimators=20, random_state=0)

In [56]:
print(get_accuracy(X_train, X_test, y_train, y_test, model))

{'test Accuracy': 0.7642857142857142, 'train Accuracy': 0.9833333333333333}


In [57]:
get_correlation(X_test, y_test,model).style.background_gradient(cmap='coolwarm')

Average correlation between predictors 0.17903958263299768


,estimator1,estimator2,estimator3,estimator4,estimator5,estimator6,estimator7,estimator8,estimator9,estimator10,estimator11,estimator12,estimator13,estimator14,estimator15,estimator16,estimator17,estimator18,estimator19,estimator20
estimator1,1.000000,0.234840,0.030549,-0.070571,0.232846,0.122908,0.322649,0.079793,0.326543,0.226010,0.276527,0.115588,0.296010,0.073807,0.151420,0.262781,0.155320,0.248726,0.246952,0.282285
estimator2,0.234840,1.000000,0.065692,0.204412,0.143611,0.017596,-0.070603,-0.088406,-0.017100,0.090969,-0.054763,-0.102919,0.246614,-0.127257,0.012632,0.051805,-0.080423,0.009579,0.186601,0.114230
estimator3,0.030549,0.065692,1.000000,0.089562,0.333225,0.104604,0.120482,0.036053,0.070146,0.154814,0.124943,0.048372,0.131551,0.197963,0.047843,0.272441,0.071679,0.108103,0.285611,0.238833
estimator4,-0.070571,0.204412,0.089562,1.000000,0.048477,0.027481,-0.103013,0.082877,0.061833,-0.005920,-0.035169,-0.111541,0.238833,0.062439,0.005935,0.048477,0.106166,0.096561,0.243432,-0.101190
estimator5,0.232846,0.143611,0.333225,0.048477,1.000000,0.244043,0.506499,0.056390,0.313983,0.267313,0.299846,0.430723,0.333225,0.496025,0.279024,0.413968,0.347097,0.339663,0.241179,0.472649
estimator6,0.122908,0.017596,0.104604,0.027481,0.244043,1.000000,0.313983,0.182041,0.365619,0.318694,0.275690,0.114706,0.345809,0.374495,0.224849,0.209072,0.190876,0.200016,-0.062717,0.302293
estimator7,0.322649,-0.070603,0.120482,-0.103013,0.506499,0.313983,1.000000,0.237181,0.558775,0.267313,0.299846,0.370960,0.242049,0.559588,0.067544,0.352280,0.513372,0.308944,0.148248,0.381755
estimator8,0.079793,-0.088406,0.036053,0.082877,0.056390,0.182041,0.237181,1.000000,0.250367,0.116905,0.055379,0.009175,-0.023328,0.228427,-0.106673,0.116654,0.174619,-0.106321,0.069171,-0.094716
estimator9,0.326543,-0.017100,0.070146,0.061833,0.313983,0.365619,0.558775,0.250367,1.000000,0.489510,0.208036,0.250224,0.242436,0.518631,0.053582,0.279013,0.567915,0.234844,0.218255,0.233590
estimator10,0.226010,0.090969,0.154814,-0.005920,0.267313,0.318694,0.267313,0.116905,0.489510,1.000000,0.317701,0.184327,0.303266,0.321573,0.070412,0.327577,0.215229,0.223788,0.038909,0.230870


-----------------------------------------------------------------------------END-----------------------------------------------------------------